<a href="https://colab.research.google.com/github/NyafeuKamdem/retail_demand_forcasting/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data_Prep**

##**Data Preparation Pipeline: From Drive to Clean Sample**

###**STEP 1: Import packages**

In [1]:
# Importing the needed packages.
import numpy as np                                                # For numerical operation arraywise.
import pandas as pd                                               # For Data manipulation.
import matplotlib.pyplot as plt                                   # For basic visualization's functions.
import seaborn as sns                                             # For advance visualization's functions.
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd  # pwd = 'print working directory' tels us where we are

/content


In [7]:
%cd drive/MyDrive/ # cd = 'change directory'

/content/drive/MyDrive


In [8]:
!pwd

/content/drive/MyDrive


###**STEP 2: Read all files except train.csv**

In [12]:
# STEP 3: Read all files but train.csv

df_holiday_events = pd.read_csv('/content/drive/MyDrive/favorita/holidays_events.csv')
df_items          = pd.read_csv('/content/drive/MyDrive/favorita/items.csv')
df_oil            = pd.read_csv('/content/drive/MyDrive/favorita/oil.csv')
df_stores         = pd.read_csv('/content/drive/MyDrive/favorita/stores.csv')
df_transactions   = pd.read_csv('/content/drive/MyDrive/favorita/transactions.csv')

###**STEP 3:Load stores and get Guayas store IDs**

In [28]:
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()

###**STEP 4: Read train.csv in chunks and filter for Guayas region**

In [29]:
# STEP 6: Read train.csv in chunks and filter for Guayas region
chunk_size = 10**6
filtered_chunks = []

for chunk in pd.read_csv('/content/drive/MyDrive/favorita/train.csv', chunksize = chunk_size):
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(chunk_filtered)
    del chunk

/tmp/ipython-input-3821781864.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('/content/drive/MyDrive/favorita/train.csv', chunksize = chunk_size):


###**STEP 5: Combine and sample**

In [30]:
# STEP 5: Combine and sample
df_train = pd.concat(filtered_chunks, ignore_index=True).sample(n=2_000_000).reset_index(drop=True)

In [31]:
df_train.shape

(2000000, 6)

###**STEP 6: Clean up memory and save df_train as a pickle file**

In [36]:
len(filtered_chunks)
del filtered_chunks

In [37]:
# SELECT THE TOP 3 FAMILIES IN TERMS OF NUMBER OF ITEM.

top3_family_per_item = df_items['family'].value_counts().reset_index()
top3_family_per_item = top3_family_per_item.rename(columns = {'count': 'item quantity'}, inplace = False)
top3_family_per_item = top3_family_per_item[:3] # top3_family_per_item = top3_family_per_item.head(3)
top3_family_per_item

,family,item quantity
0,GROCERY I,1334
1,BEVERAGES,613
2,CLEANING,446


In [38]:
# CREATE THE LIST OF ITEM'S IDS AND USE IT TO FILTER THE DATA FRAME.

item_ids = [x for x in df_items[df_items['family'].isin(top3_family_per_item['family'].unique())]['item_nbr']]

df_train = df_train[df_train['item_nbr'].isin(item_ids)]
df_train.shape

(1285361, 6)

In [39]:
# Path to save the file
save_path = "/content/drive/MyDrive/tsa_ms_project/df_train.pkl"

# Save DataFrame
df_train.to_pickle(save_path)
print(f"Saved to: {save_path}")

Saved to: /content/drive/MyDrive/tsa_ms_project/df_train.pkl
